# Get the tensors for each frame from three videos

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
from PIL import Image

## Get the pre-trained model

In [3]:
rs50 = torchvision.models.resnet50(pretrained=True)

## Keep only the first layers

In [4]:
# keep the first layers of net
modules=list(rs50.children())[:4]
convNet = nn.Sequential(*modules)

# fix the parameters
for p in convNet.parameters():
    p.requires_grad=False

# keep just the first layer 
mods=list(rs50.children())[:1]
conv = nn.Sequential(*mods)

# fix the parameters
for p in conv.parameters():
    p.requires_grad=False

## Get Image Features

The output is a 64 channel tensor ( x W x H) that can be sent to our network

In [7]:
transform = transforms.Compose([transforms.ToTensor()])
# transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

def image_loader(imgName):
    im = Image.open(imgName)
    im = transform(im)
    im = torch.unsqueeze(im,0)
    return im#.cuda()

fDir = '/mindhive/nklab5/users/hlk/projects/vidDNN/threeVids/origFrames/'
tDir = '/mindhive/nklab5/users/hlk/projects/vidDNN/threeVids/preProc/tripping/fTensors/'
vids = ['tripping240','tripping339','tripping432']                        

for v in vids:
    vDir = fDir + v
    fs = os.listdir(vDir)
    for f in fs:
        imPath = vDir + '/' + f
        im = image_loader(imPath)
        tens = conv(im)
        tensPath = tDir + v + '/tns' + f[1:5] + '.pt' 
        torch.save(tens,tensPath)